In [1]:
!pip3 install Pillow==4.0.0
!pip3 install torch==0.4.0

!pip3 install torchvision==0.2.1
# !pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

    100% |████████████████████████████████| 5.6MB 6.1MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
  Using cached https://files.pythonhosted.org/packages/d1/24/f53ff6b61b3d728b90934bddb4f03f8ab584a7f49299bf3bde56e2952612/Pillow-5.2.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


https://pytorch.org/docs/stable/nn.html

In [0]:
import torch 
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
hidden_size = 128
num_classes = 10
batch_size = 100
num_epochs = 5
learning_rate = 0.001

In [0]:
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# ユーティリティズ

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
      
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)

In [63]:
def run():
    #初期化
    model = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # モデルを学習
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):  
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()        
            if (i+1) % 300 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # モデルをテストする
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
        
for i in range(10):
  run()

Epoch [1/5], Step [300/600], Loss: 2.3007
Epoch [1/5], Step [600/600], Loss: 2.2788
Epoch [2/5], Step [300/600], Loss: 2.1306
Epoch [2/5], Step [600/600], Loss: 1.2227
Epoch [3/5], Step [300/600], Loss: 1.0548
Epoch [3/5], Step [600/600], Loss: 0.7454
Epoch [4/5], Step [300/600], Loss: 0.7421
Epoch [4/5], Step [600/600], Loss: 0.4303
Epoch [5/5], Step [300/600], Loss: 0.4900
Epoch [5/5], Step [600/600], Loss: 0.6791
Accuracy of the network on the 10000 test images: 85.39 %
Epoch [1/5], Step [300/600], Loss: 2.2958
Epoch [1/5], Step [600/600], Loss: 2.1042
Epoch [2/5], Step [300/600], Loss: 1.1400
Epoch [2/5], Step [600/600], Loss: 0.8936
Epoch [3/5], Step [300/600], Loss: 0.6041
Epoch [3/5], Step [600/600], Loss: 0.6492
Epoch [4/5], Step [300/600], Loss: 0.5781
Epoch [4/5], Step [600/600], Loss: 0.5448
Epoch [5/5], Step [300/600], Loss: 0.5753
Epoch [5/5], Step [600/600], Loss: 0.5757
Accuracy of the network on the 10000 test images: 86.5 %
Epoch [1/5], Step [300/600], Loss: 2.2412
Epo

In [13]:
clip = 0.25
torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

0.0

In [64]:
def run():
    #初期化
    model = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    #正則化
    clip = 0.25
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

    # モデルを学習
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):  
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()        
            if (i+1) % 300 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # モデルをテストする
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))
        
for i in range(10):
  run()

Epoch [1/5], Step [300/600], Loss: 2.2716
Epoch [1/5], Step [600/600], Loss: 1.9421
Epoch [2/5], Step [300/600], Loss: 1.1029
Epoch [2/5], Step [600/600], Loss: 0.9684
Epoch [3/5], Step [300/600], Loss: 0.6204
Epoch [3/5], Step [600/600], Loss: 0.6386
Epoch [4/5], Step [300/600], Loss: 0.5629
Epoch [4/5], Step [600/600], Loss: 0.4670
Epoch [5/5], Step [300/600], Loss: 0.5246
Epoch [5/5], Step [600/600], Loss: 0.6167
Accuracy of the network on the 10000 test images: 85.48 %
Epoch [1/5], Step [300/600], Loss: 2.2870
Epoch [1/5], Step [600/600], Loss: 2.1648
Epoch [2/5], Step [300/600], Loss: 1.1272
Epoch [2/5], Step [600/600], Loss: 0.9012
Epoch [3/5], Step [300/600], Loss: 0.7243
Epoch [3/5], Step [600/600], Loss: 0.7861
Epoch [4/5], Step [300/600], Loss: 0.5703
Epoch [4/5], Step [600/600], Loss: 0.5738
Epoch [5/5], Step [300/600], Loss: 0.3822
Epoch [5/5], Step [600/600], Loss: 0.4582
Accuracy of the network on the 10000 test images: 86.07 %
Epoch [1/5], Step [300/600], Loss: 2.1900
Ep

In [0]:
# テンソルを作る
x = torch.tensor(1, requires_grad=True)
w = torch.tensor(2, requires_grad=True)
b = torch.tensor(3, requires_grad=True)

# 計算グラフを作成する
y = w * x + b    # y = 2 * x + 3

# 勾配計算
y.backward()

# 勾配を確認
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2)
tensor(1)
tensor(1)


In [0]:
def clip_grad(v, min, max):
    v_tmp = v.expand_as(v)
    v_tmp.register_hook(lambda g: g.clamp(min, max))
    return v_tmp

In [0]:
clip_grad_value_

In [0]:
grad_clip = 1
clip_grad(x, -grad_clip, grad_clip)

TypeError: ignored

## パラメータの繰り返し可能性の勾配ノルムをクリップ

```
torch.nn.utils.clip_grad_norm_(parameters, max_norm, norm_type=2)
```

正則化。



パラメータの繰り返し可能性の勾配ノルムをクリップします。 ノルムは、あたかも単一のベクトルに連結されているかのように、すべての勾配にわたって一緒に計算されます。グラデーションはインプレースで変更されます。



返り値：パラメータの総ノルム

In [0]:
x = torch.Tensor([2, 2])
x.requires_grad = True

print(x)
y = torch.sum(x + 1)
print(y)
y.backward() #多次元配列だとエラーになる
print(x.grad)

y.backward()
print(x.grad)

y.backward()
print(x.grad)

tensor([ 2.,  2.])
tensor(6.)
tensor([ 1.,  1.])
tensor([ 2.,  2.])
tensor([ 3.,  3.])


In [0]:
# (10, 3) と (10, 2)のshapeのテンソルを作る
x = torch.randn(10, 3)
y = torch.randn(10, 2)
# x = torch.randn(10, 3, requires_grad = True)
# y = torch.randn(10, 2, requires_grad = True)

# 全結合レイヤー
linear = nn.Linear(3, 2)
# print ('w: ', linear.weight)
# print ('b: ', linear.bias)

# 損失関数と最適化関数
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# 順伝播
pred = linear(x)
# lossを計算
loss = criterion(pred, y)
# 逆伝播
loss.backward()
# 勾配の確認
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)
print('loss after 1 step optimization: ', loss.item())


# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)
print('loss after 1 step optimization: ', loss.item())

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)
print('loss after 1 step optimization: ', loss.item())

grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
loss after 1 step optimization:  1.105767011642456
grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
loss after 1 step optimization:  1.098765254020691
grad dL/dw:  tensor([[-0.0786, -0.0346,  0.6932],
        [-0.0362, -0.3848, -0.2169]])
grad dL/db:  tensor([ 0.1623, -0.0374])
loss after 1 step optimization:  1.0918428897857666


In [0]:
# (10, 3) と (10, 2)のshapeのテンソルを作る
x = torch.randn(10, 3)
y = torch.randn(10, 2)
# x = torch.randn(10, 3, requires_grad = True)
# y = torch.randn(10, 2, requires_grad = True)
torch.nn.utils.clip_grad_norm_(x, 0.1)
torch.nn.utils.clip_grad_norm_(y, 0.1)


# 全結合レイヤー
linear = nn.Linear(3, 2)
# print ('w: ', linear.weight)
# print ('b: ', linear.bias)

# 損失関数と最適化関数
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# 順伝播
pred = linear(x)
# lossを計算
loss = criterion(pred, y)
# 逆伝播
loss.backward()
# 勾配の確認
print ('grad dL/dw: ', linear.weight.grad) 
print ('grad dL/db: ', linear.bias.grad)

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())


# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

# １ステップ更新
optimizer.step()
# １ステップ更新したのでlossが変わる
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())






grad dL/dw:  tensor([[ 0.0704, -0.6308, -0.0338],
        [ 0.7090, -0.5395, -0.5156]])
grad dL/db:  tensor([ 0.6013,  0.9322])
loss after 1 step optimization:  1.6067063808441162
loss after 1 step optimization:  1.5803437232971191
loss after 1 step optimization:  1.554366111755371


In [0]:
# 
 


In [0]:
import torch
from torch.autograd import Variable
import numpy as np

x = torch.arange(1,4, requires_grad=True)
y = sum(x**2)
z = sum(x*3)

y.backward()
print(x.grad)

 # how to set x.grad = 0??
z.backward()
print(x.grad)

tensor([ 2.,  4.,  6.])
tensor([ 5.,  7.,  9.])


## 可変長のパッド付きシーケンスを含むTensorをパック

```
torch.nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=False)
```

## 可変長配列のパックされたバッチを埋め込み

```
torch.nn.utils.rnn.pad_packed_sequence(sequence, batch_first=False, padding_value=0.0, total_length=None)
```

# torch.nn.functional

## Convolution functions

## ２次元畳み込みを適応するには



```
torch.nn.functional.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) → Tensor
```


input：入力テンソル（ミニバッチ × 入力チャンネル × 縦 × 横）    
weight：フィルター形状（出力チャンネル × (入力チャンネル / グループ数) × 縦 × 横）
bias：Trueの場合バイアスを出力に追加する。デフォルトは1。    
stride：ストライド。デフォルトは1。    
padding：入力の両側にゼロパディングを追加。デフォルトは0。    
dilation：カーネル要素間の間隔。デフォルトは1。    
groups：入力チャネルから出力チャネルへの接続数。デフォルトは1。    



詳細と出力形状はtorch.nn.Conv2dを参照。torch.nn.Conv2dとの違いはweightを引数としているところです。

In [0]:
# 正方形のカーネルと等しいストライド
filters = torch.randn(8,4,3,3)
inputs = torch.randn(1,4,5,5)
F.conv2d(inputs, filters, padding=1)

tensor([[[[ -0.2662,   4.6487,  -4.0556,  -6.5859,   3.4060],
          [ -0.4128,   1.3292,   2.1728,   8.7131,   8.6663],
          [  3.8481,  -0.5142,   3.2911,  -2.3037,  -3.9220],
          [ -5.1769,  -1.7535,  -1.2485,   1.0203,  -1.3734],
          [  0.3103,  -6.6533,   3.0643,  -0.1680,   2.8330]],

         [[  1.9479,  -2.5230,  -4.1921,  -7.0740,  -7.5119],
          [  0.7789,  -7.8027,  -4.6254,   2.1217,  -4.1776],
          [  0.3863,  -1.6327,   3.7620,   7.4446,   1.8560],
          [ -0.4148,   6.9352,  -2.7950,  -7.3415,  -1.5851],
          [ -1.0244,  -9.4098,  -0.1047,  -0.5958,  -2.6624]],

         [[  0.2955,   1.1379,  -1.9832,   1.5713,  -2.8811],
          [ -1.6829,  -0.9123,   2.1475,  -6.9832,   0.1714],
          [ -1.2852,   2.4207,  -1.0330,  10.2498,   7.7130],
          [  3.1644,  -2.9971,  10.3442,  -0.4980,  -0.8841],
          [ -3.2795,   0.6938,   0.9730,   1.7423,  -0.5638]],

         [[  1.8431,   3.5672,  -4.0132,  -1.1640,   0.3819],
  

##Pooling functions

## 2次元アベレージプーリング


```
torch.nn.functional.avg_pool2d(input, kernel_size, stride=None, padding=0, ceil_mode=False, count_include_pad=True) → Tensor
```

input：入力テンソル（ミニバッチ × チャンネル × 縦 × 横）    
kernel_size：カーネルサイズ。単一の数値またはタプル（kH × kWkH × kW）

stride：ストライド。デフォルトはkernel_size    
padding：両端にゼロパディングを追加    
ceil_mode：Trueの場合、floorの代わりにceilを使用して出力を計算します。floorは切り捨て。ceilは切り捨て。    
count_include_pad：Trueの場合、平均化計算にゼロパディングが含まれます。デフォルト：True


In [0]:
input = torch.tensor([[[1,2,3,4,5,6], [1,2,3,4,5,6]]], dtype=torch.float)
F.avg_pool2d(input, kernel_size=(2,2), stride=2)

tensor([[[ 1.5000,  3.5000,  5.5000]]])

# torch.nn.init

## 一様分布で初期化するには

入力Tensorを一様分布 U（a、b）から引き出された値で埋めます。


```
torch.nn.init.uniform_(tensor, a=0, b=1)
```

tensor：入力テンソル    
a：一様分布の下限    
b：一様分布の上限




In [0]:
w = torch.empty(3, 5)
nn.init.uniform_(w)

tensor([[ 0.7945,  0.2324,  0.8830,  0.4259,  0.4476],
        [ 0.7428,  0.4185,  0.4235,  0.2564,  0.2008],
        [ 0.9125,  0.5551,  0.9719,  0.3061,  0.7314]])

## 正規分布で初期化するには


入力Tensorを正規分布 N（平均、標準偏差）から引き出された値で塗りつぶします。


```
torch.nn.init.normal_(tensor, mean=0, std=1)
```


tensor：入力テンソル    
mean：正規分布の平均    
std：正規分布の標準偏差    


In [0]:
w = torch.empty(3, 5)
nn.init.normal_(w)

tensor([[-1.0958, -0.5216,  0.5568,  0.2894,  0.6269],
        [ 0.1691,  2.0882,  0.7058, -2.1592,  1.4674],
        [-0.9148,  0.7325,  0.2636, -0.1951,  0.2942]])

## 指定した値で初期化するには

入力Tensorを値valで埋めます。

```
torch.nn.init.constant_(tensor, val)
```

tensor：入力テンソル    
val：テンソルを満たす値    


In [0]:
w = torch.empty(3, 5)
nn.init.constant_(w, 0.3)

tensor([[ 0.3000,  0.3000,  0.3000,  0.3000,  0.3000],
        [ 0.3000,  0.3000,  0.3000,  0.3000,  0.3000],
        [ 0.3000,  0.3000,  0.3000,  0.3000,  0.3000]])

## 単位行列で初期化するには

2次元入力Tensorを単位行列で埋めるには


```
torch.nn.init.eye_(tensor)
```

tensor：２次元の入力テンソル    


Linearレイヤーの入力のアイデンティティーを保持します。できるだけ多くの入力が保存されます。



In [0]:
w = torch.empty(3, 5)
nn.init.eye_(w)

tensor([[ 1.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.]])

## ディラックデルタ関数で{3,4,5}次元の入力テンソルを満たす？？？


TODO：理解不能



```
torch.nn.init.dirac_(tensor)
```

可能な限り多くの入力チャンネルが保存される畳み込みレイヤーの入力のアイデンティティを保持します。

In [0]:
w = torch.empty(3, 16, 5, 5)
nn.init.dirac_(w)

tensor([[[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         ...,

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0

## 与えられた非線形関数の推奨ゲイン値を得るには



```
torch.nn.init.calculate_gain(nonlinearity, param=None)
```

nonlinearity：非線形関数（nn.functional name）    
param：非線形関数のオプションパラメータ    


In [0]:
gain = nn.init.calculate_gain('linear')
print(gain)
gain = nn.init.calculate_gain('conv2d')
print(gain)
gain = nn.init.calculate_gain('relu')
print(gain)
gain = nn.init.calculate_gain('leaky_relu')
print(gain)

1
1
1.4142135623730951
1.4141428569978354


## Glorot初期化を使用して一様分布の初期化をするには



```
torch.nn.init.xavier_uniform_(tensor, gain=1)
```

tensor：n次元の入力テンソル    
gain：スケーリングファクター    



詳細は“Understanding the difficulty of training deep feedforward neural networks” - Glorot, X. & Bengio, Y. 

結果として得られるテンソルは一様分布になる。


In [0]:
w = torch.empty(3, 5)
nn.init.xavier_uniform_(w, gain=nn.init.calculate_gain('relu'))

tensor([[-0.0221,  0.4529, -1.0402,  0.9492, -1.0725],
        [-0.9870, -1.0100, -0.3421,  0.9241,  0.6219],
        [-1.1718, -1.2070,  0.7530,  0.8157,  0.9457]])

## Glorot初期化を使用して正規分布の初期化をするには


```
torch.nn.init.xavier_normal_(tensor, gain=1)
```

tensor：n次元の入力テンソル    
gain：スケーリングファクター    


詳細は“Understanding the difficulty of training deep feedforward neural networks” - Glorot, X. & Bengio, Y. 

結果として得られるテンソルは正規分布になる。



In [0]:
w = torch.empty(3, 5)
nn.init.xavier_normal_(w)

tensor([[-0.0776,  0.1208, -0.2579, -0.3210, -0.3336],
        [-0.3571, -0.1586, -0.8465,  0.2294,  0.6384],
        [-0.4535, -0.8314,  0.4807, -0.5215,  0.2322]])

## He初期化を使用して一様分布の初期化をするには



```
torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
```

tensor：n次元の入力テンソル    
a：この層の後に使用されるReLUの負の傾き（デフォルトではReLUの場合は0）
mode：'fan_in'（デフォルト）または 'fan_out'のいずれかです。 fan_inを選択すると、フォワードパスのウェイトの分散の大きさが保持されます。 fan_outを選択すると、後方パスのマグニチュードが保持されます。    
nonlinearity：'relu'または 'leaky_relu'（デフォルト）を使用することを推奨する非線形関数（nn.functional name）。    



詳細は“Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification” - He, K. et al. (2015)



In [0]:
w = torch.empty(3, 5)
nn.init.kaiming_uniform_(w, mode='fan_in', nonlinearity='relu')

tensor([[-0.6863,  0.0687,  0.5840,  0.6194,  0.6268],
        [-0.5025, -0.2338,  0.2653,  0.4436, -0.6380],
        [-0.9431,  0.2097,  0.3742, -0.7311,  0.5842]])

## He初期化を使用して正規分布の初期化をするには


```
torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
```

tensor：n次元の入力テンソル    
a：この層の後に使用されるReLUの負の傾き（デフォルトではReLUの場合は0）
mode：'fan_in'（デフォルト）または 'fan_out'のいずれかです。 fan_inを選択すると、フォワードパスのウェイトの分散の大きさが保持されます。 fan_outを選択すると、後方パスのマグニチュードが保持されます。    
nonlinearity：'relu'または 'leaky_relu'（デフォルト）を使用することを推奨する非線形関数（nn.functional name）。    



詳細は“Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification” - He, K. et al. (2015)




In [0]:
w = torch.empty(3, 5)
nn.init.kaiming_normal_(w, mode='fan_out', nonlinearity='relu')

tensor([[ 0.4191,  0.7646, -0.1674, -1.4745, -0.3846],
        [-1.2578,  0.5891, -0.1393, -0.0649,  1.0568],
        [-0.3875,  0.0445,  0.4014,  0.4188,  0.5702]])

## 直交行列で初期化するには？？？


入力テンソルを（半）直交行列で満たす


```
torch.nn.init.orthogonal_(tensor, gain=1)
```

tensor：２次元以上のn次元の入力テンソル    
gain：スケーリングファクター    


詳細は“Exact solutions to the nonlinear dynamics of learning in deep linear neural networks” - Saxe, A. et al. (2013)。



入力テンソルは少なくとも2次元でなければならず、2次元以上のテンソルでは後続次元が平坦化される。


In [0]:
w = torch.empty(3, 5)
nn.init.orthogonal_(w)

tensor([[ 0.0655, -0.7997, -0.0716, -0.5422, -0.2389],
        [ 0.4006, -0.4301, -0.1266,  0.7786, -0.1797],
        [-0.8761, -0.2986,  0.2087,  0.3153, -0.0190]])

## 非ゼロ要素の正規分布疎行列で初期化するには


```
torch.nn.init.sparse_(tensor, sparsity, std=0.01)
```

tensor：n次元の入力テンソル    
sparsity：ゼロに設定される各列の要素の割合    
std：非ゼロ値を生成するために使用される正規分布の標準偏差    


詳細は“Deep learning via Hessian-free optimization” - Martens, J. (2010).

非ゼロ要素が正規分布（0,0.01）N（0,0.01）から引き出される、疎行列として2D入力Tensorを埋めます。 、J.（2010）。


In [0]:
w = torch.empty(3, 5)
nn.init.sparse_(w, sparsity=0.1)

tensor(1.00000e-02 *
       [[-0.4569,  0.0000, -0.5139,  0.7227,  2.3415],
        [-0.3503, -1.0910,  0.0000,  0.0000, -0.6600],
        [ 0.0000,  1.8526,  0.2404,  0.0212,  0.0000]])